In [1]:
import pandas as pd
import numpy as np

In [2]:
data_location = "C:\\Users\\User\\Documents\\4. Fourth Year\\Project\\Railway-Fares\\Data\\"
output_location = "C:\\Users\\User\\Documents\\4. Fourth Year\\Project\\Railway-Fares\Data\\Cleansed Data\\"

In [3]:
# Read in all files needed

station_master = pd.read_csv(output_location + "station.csv", dtype = str)
simd_lookup = pd.read_csv(data_location + "SIMD Postcode Lookup.csv", dtype = str)
simd = pd.read_csv(data_location + "simd2020_withinds.csv", dtype = str)


clusters = pd.read_csv(data_location + r"Parsed Fare Data\Station_Cluster.csv", dtype = str)
fare = pd.read_csv(data_location + r"Parsed Fare Data\Fare.csv", dtype = str)
flow = pd.read_csv(data_location + r"Parsed Fare Data\Flow.csv", dtype = str)
ticket_types = pd.read_csv(data_location + r"Parsed Fare Data\Ticket_Types.csv", dtype = str)

In [4]:
def fix_NLC(nlc):
    length = len(nlc)
    if length < 4:
        return "".join(['0' for i in range(4-length)]) + nlc
    else:
        return nlc
    
station_master["NLC"] = station_master["NLC"].apply(fix_NLC)
        

In [5]:
flow = flow[["ORIGIN_CODE", "DESTINATION_CODE", "ROUTE_CODE", "DIRECTION", "END_DATE", "START_DATE", "FLOW_ID"]]
flow_inscope = flow.merge(station_master[["NLC", "Name"]], left_on = "ORIGIN_CODE", right_on = "NLC")
flow_inscope = flow_inscope.merge(station_master[["NLC", "Name"]], left_on = "DESTINATION_CODE", right_on = "NLC", suffixes = ("", "_dest"))

In [6]:
fares_out = fare.merge(flow_inscope, on = "FLOW_ID")
fares_out = fares_out[["TICKET_CODE", "FARE", "RESTRICTION_CODE", "ORIGIN_CODE", "DESTINATION_CODE", "DIRECTION", "END_DATE", "START_DATE"]]

In [7]:
ticket_types_inscope = ticket_types[(ticket_types["TKT_GROUP"] == 'S') & (ticket_types["TKT_TYPE"] == 'S')][["TICKET_CODE", "DESCRIPTION"]]

In [8]:
fares_inscope = fares_out.merge(ticket_types_inscope, on = "TICKET_CODE")[["TICKET_CODE", "FARE", "RESTRICTION_CODE", "ORIGIN_CODE", "DESTINATION_CODE", "DIRECTION", "END_DATE", "START_DATE"]]

In [9]:
station_datazone = station_master.merge(simd_lookup, how = 'left', on = "Postcode")
station_datazone = station_datazone[["NLC", "DZ"]]
datazone_info = simd[["Data_Zone", "Intermediate_Zone", "Council_area"]]
station_datazone = station_datazone.merge(datazone_info, how = 'left', left_on = "DZ", right_on = "Data_Zone")
station_datazone = station_datazone[["NLC"] + list(datazone_info.columns)]

In [10]:
# Output final files
fares_inscope.to_csv(output_location + "fares.csv", index = False)
ticket_types.to_csv(output_location + "ticket_types.csv", index = False)
station_datazone.to_csv(output_location + "station_datazone.csv", index = False)